In [1]:
import geopandas as gpd
import momepy as mm
from libpysal.graph import Graph, read_parquet

In [2]:
region_id = 69333


buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'

In [26]:
buildings_dir = '/data/uscuni-ulce/processed_data/simplified_buildings/'

In [31]:
buildings.reset_index(drop=True).to_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')

In [27]:
buildings = gpd.read_parquet(buildings_dir + f"buildings_{region_id}.parquet")

In [7]:
buildings = gpd.read_parquet(buildings_dir + f"buildings_{region_id}.parquet")
buildings_q1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")

In [8]:
# connected buildings count


In [9]:
buildings_q1._adjacency

focal   neighbor
0       0           1
1       1           1
        3           1
2       2           1
        19          1
                   ..
457120  457120      1
457121  457121      1
457122  457122      1
457123  457123      1
457124  457124      1
Name: weight, Length: 653007, dtype: int64

In [10]:
from shapely import GeometryCollection
from shapely import unary_union

In [8]:
connected_buildings = buildings.geometry.groupby(buildings_q1.component_labels).apply( lambda x: unary_union(x.values))

In [9]:
connected_buildings = connected_buildings.set_crs(epsg=3035).buffer(.1).normalize().make_valid()

In [10]:
connected_buildings.is_valid.all()

np.True_

In [11]:
connected_buildings.length

component labels
0          50.265274
1          92.868932
2          87.794672
3          66.110550
4         104.499251
             ...    
367097     25.337530
367098     15.981284
367099     36.778758
367100     43.070813
367101     23.713898
Length: 367102, dtype: float64

In [12]:
buildings_q1.component_labels

focal
0              0
1              1
2              2
3              1
4              3
           ...  
457120    367097
457121    367098
457122    367099
457123    367100
457124    367101
Name: component labels, Length: 457125, dtype: int32

In [13]:
import pandas as pd


comps = buildings_q1.component_labels
cardinalities = comps.value_counts()
lens = connected_buildings.length
areas = connected_buildings.area

In [14]:
%%time
elos = mm.elongation(connected_buildings)
eris = mm.equivalent_rectangular_index(connected_buildings)
ccos = mm.circular_compactness(connected_buildings)
lals = mm.longest_axis_length(connected_buildings)

CPU times: user 58.8 s, sys: 10.8 ms, total: 58.8 s
Wall time: 58.8 s


In [15]:
buildings.loc[comps.index.values, 'mibCou'] = cardinalities.loc[comps.values].values

buildings.loc[comps.index.values, 'mibLen'] = lens.loc[comps.values].values

buildings.loc[comps.index.values, 'mibAre'] = areas.loc[comps.values].values

In [16]:
buildings.loc[comps.index.values, 'mibElo'] = elos.loc[comps.values].values

buildings.loc[comps.index.values, 'mibERI'] = eris.loc[comps.values].values

buildings.loc[comps.index.values, 'mibCCo'] = ccos.loc[comps.values].values

buildings.loc[comps.index.values, 'mibLAL'] = lals.loc[comps.values].values


In [41]:
fr = mm.facade_ratio(connected_buildings)
buildings.loc[comps.index.values, 'mibFR'] = fr.loc[comps.values].values

In [42]:
sco = mm.square_compactness(connected_buildings)
buildings.loc[comps.index.values, 'mibSCO'] = sco.loc[comps.values].values


In [55]:
# fd =  mm.fractal_dimension(connected_buildings)
# buildings.loc[comps.index.values, 'mibFD'] = fd.loc[comps.values].values


In [45]:
# sq =  mm.squareness(connected_buildings)
# buildings.loc[comps.index.values, 'mibSqu'] = sq.loc[comps.values].values

In [14]:
plotting = buildings[buildings.area > 1_000]

In [15]:
import lonboard


layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.3)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [16]:
from sidecar import Sidecar
sc = Sidecar(title='buildings')
m = lonboard.Map(layer)
with sc:
    display(m)

In [58]:
column = 'mibSCO'

In [59]:
from mapclassify import classify
import matplotlib as mpl
from lonboard.colormap import apply_continuous_cmap
import numpy as np

classifier = classify(plotting[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

In [34]:
buildings_q1[179235]

neighbor
178869    1
179235    1
Name: weight, dtype: int64

In [70]:
### add connected buildings buildings shapes
buildings = gpd.read_parquet(buildings_dir + f"buildings_{region_id}.parquet")
buildings_q1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")

connected_buildings = buildings_q1.apply(
    buildings.geometry.values, lambda x: GeometryCollection(x.values)
)
connected_buildings = gpd.GeoSeries(connected_buildings)

buildings['mibCou'] = buildings_q1.cardinalities
buildings['mibElo'] = mm.elongation(connected_buildings)
buildings['mibERI'] = mm.equivalent_rectangular_index(connected_buildings)
buildings['mibCCo'] = mm.circular_compactness(connected_buildings)
buildings['mibLAL'] = mm.longest_axis_length(connected_buildings)


focal
0         0.506705
1         0.658297
2         0.753207
3         0.658297
4         0.304004
            ...   
457120    0.280867
457121    0.719019
457122    0.477450
457123    0.582264
457124    0.793982
Name: elongation, Length: 457125, dtype: float64